In [9]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pydicom
import tensorflow as tf

In [10]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, Conv2D, MaxPooling2D
from keras.callbacks import CSVLogger
from livelossplot.keras import PlotLossesCallback

In [11]:
def load_images_from_folder(folder):
    images = []
    for i in range(1,601):
        filename = "Mask ("+str(i)+").tif"
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [12]:
masks = load_images_from_folder("mask")

In [13]:
def load_dicom_images_from_folder(folder):
    images = []
    for i in range(1,601):
        filename = "Image ("+str(i)+").dcm"
        data = pydicom.dcmread(os.path.join(folder,filename))
        img = data.pixel_array
        if img is not None:
            images.append(img)
    return images

In [14]:
images = load_dicom_images_from_folder("images")

In [16]:
plt.imshow(images[179], cmap=plt.cm.bone)
plt.show()

In [17]:
maskg = []
for i in range(600):    
    maskg.append(cv2.cvtColor(masks[i], cv2.COLOR_BGR2GRAY))

In [18]:
idx = []
count = 0
for i in range(600):
    if(images[i].shape != maskg[i].shape):
        idx.append(i)
        count+=1
print(idx,count)

([], 0)


In [19]:
ildimg = []
for i in range(600): 
    if i not in idx:
        ildimg.append(np.multiply(images[i],maskg[i]))

In [20]:
def crop(img, tol=0):
    mask = img>tol
    m,n = img.shape
    mask0,mask1 = mask.any(0),mask.any(1)
    col_start,col_end = mask0.argmax(), n-mask0[::-1].argmax()
    row_start,row_end = mask1.argmax(),m-mask1[::-1].argmax()
    cropped = img[row_start:row_end,col_start:col_end]
    #final = img.reshape(cropped,(512,512))
    return cropped

In [21]:
croppi = []
for i in range (600):
    #print(i)
    a = crop(ildimg[i])
    croppi.append(a)
    
croppi = np.asarray(croppi)

In [22]:
final = []
for i in range(600):
    croppi[i] = np.asarray(croppi[i])
    final.append(cv2.resize(croppi[i],(256,256),interpolation=cv2.INTER_AREA))

In [23]:
final = np.asarray(final)
final.shape

(600, 256, 256)

In [24]:
target = []
for i in range(6):
    for j in range(i*100,(i+1)*100):
        target.append(i)
target = np.asarray(target)

In [25]:
from sklearn.model_selection import train_test_split
set1x, set2x, set1t, set2t = train_test_split(final, target, test_size=0.5, random_state=10)
set1x = np.asarray(set1x)
set1t = np.asarray(set1t)
set2x = np.asarray(set2x)
set2t = np.asarray(set2t)

In [26]:
set1x = set1x.reshape(set1x.shape[0], 256, 256,1)
set2x = set2x.reshape(set2x.shape[0], 256, 256,1)

In [27]:
set1x = set1x.astype('float32')
set2x = set2x.astype('float32')
set1x /= 255
set2x /= 255

In [28]:
from keras.utils import np_utils
set1t = np_utils.to_categorical(set1t, 6)
set2t = np_utils.to_categorical(set2t, 6)
set1t.shape

(300, 6)

In [29]:
classifier = Sequential()

In [30]:
classifier.add(Conv2D(filters = 56,kernel_size = (3,3), activation = 'relu', input_shape = (256,256,1)))
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())

classifier.add(Dense(units = 64, activation = 'relu'))

classifier.add(Dense(units = 6 , activation = 'softmax'))

In [31]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy','accuracy'])

In [32]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 56)      560       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 56)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 32)      16160     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 123008)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                7872576   
_________________________________________________________________
dense_2 (Dense)              (None, 6)                

In [ ]:
classifier.fit(set1x,set1t, epochs = 1, steps_per_epoch = 3)

Epoch 1/1
